In [ ]:
from Training import data_spilt
from Training.training_helper_rm import generate_rm_test_data
from Training.NetWorks import RewardModel
from monai.networks.nets import SwinUNETR
import torch

from monai.data import (
    Dataset,
    DataLoader,
)

import matplotlib.pyplot as plt

batch_size=6
num_epoch=1000
device = 'cuda:1'

seg_list, rl_list, holdout_list = data_spilt('/home/xiangcen/RLModality/picai_h5', 110, 100, 10)
seg_list_promise, rl_list_promise, holdout_list_promise = data_spilt('/home/xiangcen/RLModality/promise_h5', 231, 180, 20)

train_list = rl_list + rl_list_promise
test_list = holdout_list + holdout_list_promise


train_loader = DataLoader(Dataset(train_list), batch_size=2, shuffle=True, drop_last=True)
test_loader = DataLoader(Dataset(test_list), batch_size=2, shuffle=True, drop_last=False)


seg_model = SwinUNETR(
    img_size = (128, 128, 32),
    in_channels = 2,
    out_channels = 1,
    depths = (2, 2, 2, 2),
    num_heads = (3, 6, 12, 24),
    drop_rate = 0.1,
    attn_drop_rate = 0.1,
    dropout_path_rate = 0.1,
    downsample="mergingv2",
    use_v2=True,
)
seg_model.load_state_dict(torch.load("/home/xiangcen/RLModality/models/segmentation.ptm", map_location=device, weights_only=True))
rm_model = RewardModel()
rm_model.load_state_dict(torch.load('/home/xiangcen/RLModality/models/rm.ptm', map_location=device, weights_only=True))
rm_model.eval()







def visualize(index):
    
    
    batch = [test_list[index]]
    print(batch)
    
    img, accuracy = generate_rm_test_data(test_list[index], 20, seg_model)
    # img[2, ...] = 0
    reward = rm_model(img.float().unsqueeze(0)).item()
    print(accuracy, reward)
    
    rows, cols = 4, 8
    fig, axes = plt.subplots(rows, cols, figsize=(12, 6))

    # Loop through images and plot each one in the grid
    for i, ax in enumerate(axes.flat):
        ax.imshow(img[2, :, :, i], cmap='gray')  # Change 'cmap' as needed
        ax.axis('off')  # Turn off axis labels

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.show()

In [ ]:
visualize(5)